In [226]:
from ortools.sat.python import cp_model
import numpy as np

In [227]:
# Declare the model
model = cp_model.CpModel()

In [228]:
# Define constant variables

T = 105 # Time step number
P = 6 # h 
Column = 4
Row = 14
numNode= Row * Column # Number of nodes

#startPostions = [1,2,3,4,5,6,7,8,9,10]
startPostions = [0,1,2,3,4,5]
#R = [0,28,52,80,116,3,31,55,83,119]
R = [0,28,52,3,31,55]
goalPostions = [[R[0],R[0],R[1]],[R[1],R[1],R[2]],[R[2],R[2],R[3]],[R[3],R[3],R[4]],[R[4],R[4],R[5]],[R[5],R[5],R[0]]]
targetTimes = [[5,55,94],[5,55,93],[5,55,104],[5,55,94],[5,55,93],[5,55,104]]
processingTime = [50,28,48]

In [229]:
# Define decision variables
x = [[[[model.NewIntVar(0,1,f'x_{i}_{j}_{t}_{k}') for k in range(P)] for t in range(T)] for j in range(numNode)] for i in range(numNode)]

In [230]:
# Define intermediate variables
y = [[model.NewIntVar(0,1,f'y_{k}_{t}') for t in range(T)] for k in range(P)]

In [231]:
# N_i is the set of nodes connected to node i
Map = np.arange(Row*Column).reshape(Row, Column)
N = [[] for i in range(Row*Column)]

# D_i is the set of the nodes connected to node i on the diagonal
D = [[] for i in range(Row*Column)]

# Four corners
'''
corners = [0,3,116,119]
N[corners[0]]=[1,4,5]
N[corners[1]]=[2,6,7]
N[corners[2]]=[112,113,117]
N[corners[3]]=[114,115,118]
'''
corners = [0,3,52,55]
N[corners[0]]=[corners[0],1,4,5]
D[corners[0]]=[corners[0],5]
N[corners[1]]=[corners[1],2,6,7]
D[corners[1]]=[corners[1],6]
#N[corners[2]]=[corners[2],112,113,117]
N[corners[2]]=[corners[2],48,49,53]
D[corners[2]]=[corners[2],49]
#N[corners[3]]=[corners[3],114,115,118]
N[corners[3]]=[corners[3],50,51,54]
D[corners[2]]=[corners[3],50]

# Nodes in the edge
edgeNodesB = [1,2]
edgeNodesT = [53,54]
edgeNodesL = []
edgeNodesR = []
for i in range(1,Row-1):
    edgeNodesL.append(Map[i][0])
    edgeNodesR.append(Map[i][Column-1])

for node in edgeNodesB:
    N[node]=[node,node-1,node+1,node+Column-1,node+Column,node+Column+1]
    D[node]=[node+Column-1,node+Column+1]

for node in edgeNodesT:
    N[node]=[node,node-1,node+1,node-Column-1,node-Column,node-Column+1]
    D[node]=[node-Column-1,node-Column+1]

for node in edgeNodesL:
    N[node]=[node,node-Column,node-Column+1,node+1,node+Column,node+Column+1]
    D[node]=[node-Column+1,node+Column+1]

for node in edgeNodesR:
    N[node]=[node,node-Column,node-Column-1,node-1,node+Column,node+Column-1]
    D[node]=[node-Column-1,node+Column-1]
    
# Nodes in the center part
edgeNodes = corners+edgeNodesB+edgeNodesT+edgeNodesL+edgeNodesR
nodes = list(range(Row*Column))
centerNodes = list(set(nodes)-set(edgeNodes))

for node in centerNodes:
    N[node]=[node,node-Column-1,node-Column,node-Column+1,node-1,node+1,node+Column-1,node+Column,node+Column+1]
    D[node]=[node-Column-1,node-Column+1,node+Column-1,node+Column+1]

In [232]:
list(set(N[5])-set(D[5]))

[1, 4, 5, 6, 9]

In [233]:
# Define constraints

# Definition of x
for k in range(P):
    for t in range(T):
        model.Add(sum(x[i][j][t][k] for j in range(numNode) for i in range(numNode)) == 1)
        for i in range(numNode):
            model.Add(sum(x[i][j][t][k] for j in range(numNode) if j not in N[i]) == 0)
            #model.Add(sum(x[i][j][t][k] for j in range(numNode) if j in N[j]) <= 1)

In [234]:
for i in range(numNode):
    for t in range(T-1):
        for k in range(P):
            model.Add(sum(x[j][i][t][k] for j in range(numNode) if j in N[i]) == sum(x[i][n][t+1][k] for n in range(numNode) if n in N[i]))

In [235]:
# Collision avoidance
for j in range(numNode):
    for t in range(T):
        model.Add(sum(x[i][j][t][k] for k in range(P) for i in range(numNode) if i in N[j])<=1)

In [236]:
for t in range(T):
    for i in range(numNode):
        for j in range(numNode):
            if i!=j:
                model.Add(sum((x[i][j][t][k]+x[j][i][t][k]) for k in range(P)) <= 1)

In [237]:
# Movement Constraints
diagFlag = [[[model.NewBoolVar(f'diagFlag_{i}_{t}_{k}') for k in range(P)] for t in range(T-2)] for i in range(numNode)]
for i in range(numNode):
    for k in range(P):
        model.Add(diagFlag[i][2][k]==sum(x[i][n][t][k] for n in range(numNode) if n in D[i]))
        model.Add(sum(x[j][j][3][k] for j in D[i])==1).OnlyEnforceIf(diagFlag[i][2][k])
        model.Add(sum(x[j][j][4][k] for j in D[i])==1).OnlyEnforceIf(diagFlag[i][2][k])
        for t in range(2,T-2):
            model.Add(diagFlag[i][t][k]<=sum(x[j][i][t-1][k] for j in range(numNode) if j in list(set(N[i])-set(D[i]))))
            model.Add(diagFlag[i][t][k]<=sum(x[i][n][t][k] for n in range(numNode) if n in D[i]))
            model.Add(diagFlag[i][t][k]>=sum(x[j][i][t-1][k] for j in range(numNode) if j in list(set(N[i])-set(D[i])))+sum(x[i][n][t][k] for n in range(numNode) if n in D[i])-1)

            model.Add(sum(x[j][j][t+1][k] for j in D[i])==1).OnlyEnforceIf(diagFlag[i][t][k])
            model.Add(sum(x[j][j][t+2][k] for j in D[i])==1).OnlyEnforceIf(diagFlag[i][t][k])
        



In [238]:
# Start Postions
for k in range(P):
    model.Add(x[R[k]][R[k]][0][k]==1)


In [239]:
# Completion conditions
for pallet in range(P):
    for task in range(len(goalPostions[0])):
        goal = goalPostions[pallet][task]
        timing = targetTimes[pallet][task]
        model.Add(sum(x[i][goal][timing][pallet] for i in range(numNode) if i in N[goal])==1)
        if task<2:
            for t in range(timing+1,timing+processingTime[task]):
                model.Add(x[goal][goal][t][pallet]==1)
    for t in range(T):
        model.Add(sum(x[i][goal][t][pallet] for i in range(numNode) if i in N[goal])==1-y[pallet][t])
        if t<=timing-1:
            model.Add(-y[pallet][t]+y[pallet][t+1]<=0)


In [240]:
# Processing Time Window


In [241]:
# Objective Function
cost = sum(y[k][t] for t in range(T) for k in range(P))
model.Minimize(cost)

In [242]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Maximum of objective function: {solver.ObjectiveValue()}\n')
else:
    print('No solution found.')

Maximum of objective function: 558.0



In [243]:
#for pallet in range(P):
route = [[] for k in range(P)]
for pallet in range(P):
    for i in range(numNode): 
        for j in range(numNode):
            for t in range(T):
                if solver.Value(x[i][j][t][pallet]) == 1:
                    route[pallet].append((i,j,t))
                
    route[pallet]=sorted(route[pallet],key= lambda item: item[2])

In [244]:
#route=sorted(route,key= lambda item: item[2])

In [245]:
route[0]

[(0, 0, 0),
 (0, 5, 1),
 (5, 1, 2),
 (1, 0, 3),
 (0, 1, 4),
 (1, 0, 5),
 (0, 0, 6),
 (0, 0, 7),
 (0, 0, 8),
 (0, 0, 9),
 (0, 0, 10),
 (0, 0, 11),
 (0, 0, 12),
 (0, 0, 13),
 (0, 0, 14),
 (0, 0, 15),
 (0, 0, 16),
 (0, 0, 17),
 (0, 0, 18),
 (0, 0, 19),
 (0, 0, 20),
 (0, 0, 21),
 (0, 0, 22),
 (0, 0, 23),
 (0, 0, 24),
 (0, 0, 25),
 (0, 0, 26),
 (0, 0, 27),
 (0, 0, 28),
 (0, 0, 29),
 (0, 0, 30),
 (0, 0, 31),
 (0, 0, 32),
 (0, 0, 33),
 (0, 0, 34),
 (0, 0, 35),
 (0, 0, 36),
 (0, 0, 37),
 (0, 0, 38),
 (0, 0, 39),
 (0, 0, 40),
 (0, 0, 41),
 (0, 0, 42),
 (0, 0, 43),
 (0, 0, 44),
 (0, 0, 45),
 (0, 0, 46),
 (0, 0, 47),
 (0, 0, 48),
 (0, 0, 49),
 (0, 0, 50),
 (0, 0, 51),
 (0, 0, 52),
 (0, 0, 53),
 (0, 0, 54),
 (0, 0, 55),
 (0, 0, 56),
 (0, 0, 57),
 (0, 0, 58),
 (0, 0, 59),
 (0, 0, 60),
 (0, 0, 61),
 (0, 0, 62),
 (0, 0, 63),
 (0, 0, 64),
 (0, 0, 65),
 (0, 0, 66),
 (0, 0, 67),
 (0, 0, 68),
 (0, 0, 69),
 (0, 0, 70),
 (0, 0, 71),
 (0, 0, 72),
 (0, 0, 73),
 (0, 0, 74),
 (0, 0, 75),
 (0, 0, 76),
 (0, 0, 7

In [246]:
Map

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31],
       [32, 33, 34, 35],
       [36, 37, 38, 39],
       [40, 41, 42, 43],
       [44, 45, 46, 47],
       [48, 49, 50, 51],
       [52, 53, 54, 55]])